<a href="https://colab.research.google.com/github/MuhammadHasbiAshshiddieqy/pythonDS-COVID-19-/blob/master/Covid_With_Bokeh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check my **GitHub** : **https://github.com/MuhammadHasbiAshshiddieqy/**

In [267]:
!pip install coronavirus

In [305]:
!pip install pyproj

     |████████████████████████████████| 10.4MB 4.9MB/s 


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta as td

%matplotlib inline

import asyncio
import aiohttp

from coronavirus import get_cases

# Get data from coronavirus API

In [0]:
async def getData():
  session = aiohttp.ClientSession()
  cases = await get_cases(session)
  country,confirmed,deaths,recovered,latitude,longitude = [],[],[],[],[],[]
  for case in cases:
    #print(f"Cases in {case.country}: {case.confirmed}")
    #attr of case = id, country, confirmed, deaths, recovered, latitude, longitude, updated
    country.append(case.country)
    confirmed.append(case.confirmed)
    deaths.append(case.deaths)
    recovered.append(case.recovered)
    latitude.append(case.latitude)
    longitude.append(case.longitude)
    await session.close()
  df = pd.DataFrame({"Country":country,"Confirmed":confirmed,"Deaths":deaths,"Recovered":recovered,"Latitude":latitude,"Longitude":longitude})
  return df

In [270]:
  loop = asyncio.get_event_loop()
  dfCovid = loop.run_until_complete(getData())
  dfCovid.head()

,Country,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Australia,6714,83,5541,-25.0000,133.0000
1,Austria,15274,549,12362,47.5162,14.5501
2,Canada,47147,2663,16883,60.0010,-95.0010
3,China,83912,4637,78290,30.5928,114.3055
4,Denmark,8896,422,5994,56.0000,10.0000


In [271]:
dfCovid["Death Rate(%)"] = round((dfCovid["Deaths"]/dfCovid["Confirmed"])*100,2)
dfCovid["Not Recovered"] = dfCovid["Confirmed"] - dfCovid["Recovered"] - dfCovid["Deaths"]
dfCovid.head(3)

,Country,Confirmed,Deaths,Recovered,Latitude,Longitude,Death Rate(%),Not Recovered
0,Australia,6714,83,5541,-25.0000,133.0000,1.24,1090
1,Austria,15274,549,12362,47.5162,14.5501,3.59,2363
2,Canada,47147,2663,16883,60.0010,-95.0010,5.65,27601


# Get data (last data updated) from https://github.com/CSSEGISandData/COVID-19

In [0]:
date = str(dt.strftime(dt.now() - td(1), '%m-%d-%Y'))

In [0]:
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"+date+".csv"

In [0]:
CovDetail = pd.read_csv(url)
dfConfirmedTimeSeries = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
dfDeathsTimeSeries = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
dfRecoveredTimeSeries = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

In [275]:
CovDetail.head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-27 02:30:52,34.223334,-82.461707,27,0,0,27,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-27 02:30:52,30.295065,-92.414197,130,7,0,123,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-27 02:30:52,37.767072,-75.632346,187,3,0,184,"Accomack, Virginia, US"


# Prepare Data for Visualization

In [276]:
dfConfTS = pd.DataFrame(dfConfirmedTimeSeries.groupby("Country/Region")[dfConfirmedTimeSeries.columns[list(dfConfirmedTimeSeries.columns).index("Long")+1:]].sum())
dfConfTS = dfConfTS.reset_index(drop=False)
dfConfTS.sort_values(dfConfTS.columns[-1],ascending=False).head(3)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,...,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20
171,US,1,1,2,2,5,5,5,5,5,7,8,8,11,11,11,11,11,11,11,11,12,12,13,13,13,13,13,13,13,13,15,15,15,51,51,57,58,60,68,...,7783,13747,19273,25600,33276,43843,53736,65778,83836,101657,121465,140909,161831,188172,213242,243622,275367,308650,336802,366317,397121,428654,462780,496535,526396,555313,580619,607670,636350,667592,699706,732197,758809,784326,811865,840351,869170,905358,938154,965785
156,Spain,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,6,13,15,32,45,...,13910,17963,20410,25374,28768,35136,39885,49515,57786,65719,73235,80110,87956,95923,104118,112065,119199,126168,131646,136675,141942,148220,153222,158273,163027,166831,170099,172541,177644,184948,190839,191726,198674,200210,204178,208389,213024,219764,223759,226629
84,Italy,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,20,62,155,229,322,453,655,888,1128,...,35713,41035,47021,53578,59138,63927,69176,74386,80589,86498,92472,97689,101739,105792,110574,115242,119827,124632,128948,132547,135586,139422,143626,147577,152271,156363,159516,162488,165155,168941,172434,175925,178972,181228,183957,187327,189973,192994,195351,197675


In [277]:
dfDeathTS = pd.DataFrame(dfDeathsTimeSeries.groupby("Country/Region")[dfDeathsTimeSeries.columns[list(dfDeathsTimeSeries.columns).index("Long")+1:]].sum())
dfDeathTS = dfDeathTS.reset_index(drop=False)
dfDeathTS.sort_values(dfDeathTS.columns[-1],ascending=False).head(3)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,...,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20
171,US,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,118,200,244,307,417,557,706,942,1209,1581,2026,2467,2978,3874,4760,5929,7090,8408,9619,10783,12794,14704,16544,18595,20471,22029,23538,25843,28338,32930,36787,38664,40661,42094,44444,46622,49954,51949,53755,54881
84,Italy,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,7,10,12,17,21,29,...,2978,3405,4032,4825,5476,6077,6820,7503,8215,9134,10023,10779,11591,12428,13155,13915,14681,15362,15887,16523,17127,17669,18279,18849,19468,19899,20465,21067,21645,22170,22745,23227,23660,24114,24648,25085,25549,25969,26384,26644
156,Spain,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,623,830,1043,1375,1772,2311,2808,3647,4365,5138,5982,6803,7716,8464,9387,10348,11198,11947,12641,13341,14045,14792,15447,16081,16606,17209,17756,18056,18708,19315,20002,20043,20453,20852,21282,21717,22157,22524,22902,23190


In [278]:
dfRecovTS = pd.DataFrame(dfRecoveredTimeSeries.groupby("Country/Region")[dfRecoveredTimeSeries.columns[list(dfRecoveredTimeSeries.columns).index("Long")+1:]].sum())
dfRecovTS = dfRecovTS.reset_index(drop=False)
dfRecovTS.sort_values(dfRecovTS.columns[-1],ascending=False).head(3)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,...,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20
156,Spain,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...,1081,1107,1588,2125,2575,2575,3794,5367,7015,9357,12285,14709,16780,19259,22647,26743,30513,34219,38080,40437,43208,48021,52165,55668,59109,62391,64727,67504,70853,74797,74797,74797,77357,80587,82514,85915,89250,92355,95708,117727
65,Germany,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,12,12,12,14,14,14,14,14,15,16,16,16,...,105,113,180,233,266,266,3243,3547,5673,6658,8481,9211,13500,16100,18700,22440,24575,26400,28700,28700,36081,46300,52407,53913,57400,60300,64300,68200,72600,77000,83114,85400,88000,91500,95200,99400,103300,109800,109800,112000
171,US,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,3,3,3,3,3,3,3,3,3,3,5,5,5,5,6,6,6,7,7,...,105,121,147,176,178,178,348,361,681,869,1072,2665,5644,7024,8474,9001,9707,14652,17448,19581,21763,23559,25410,28790,31270,32988,43482,47763,52096,54703,58545,64840,70337,72329,75204,77366,80203,99079,100372,106988


# Start Code With Bokeh

In [0]:
# Bokeh Libraries
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import ColumnDataSource, CategoricalColorMapper, Div, HoverTool, value, LabelSet, Legend, LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.layouts import gridplot, column
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer
import warnings
warnings.filterwarnings('ignore')

In [280]:
# The figure will be rendered in a static HTML file called output_file_test.html
# output_file('output_file_test.html', title='Empty Bokeh Figure') -> If wanna use .py and output with HTML

# The figure will be right in my Jupyter Notebook
output_notebook()

# Set up a generic figure() object
fig = figure()

# See what it looks like
show(fig)

In [281]:
'''
# Notes : Use this command if ur output stacked
# Import reset_output (only needed once) 
from bokeh.plotting import reset_output

# Use reset_output() between subsequent show() calls, as needed
reset_output()
'''

'\n# Notes : Use this command if ur output stacked\n# Import reset_output (only needed once) \nfrom bokeh.plotting import reset_output\n\n# Use reset_output() between subsequent show() calls, as needed\nreset_output()\n'

In [0]:
def startDate(idxs):
  if idxs > 0:
    return idxs-1
  else:
    return idxs

In [283]:
TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom,hover'

# Assign Country Name
Country = "Indonesia"

# Assign n last day
# Assign 1 to get data from all dates
day = 1

# Change Data Type of Dates to prevent string ordered
dates_list = [dt.strptime(date, "%m/%d/%y").date() for date in list(dfConfTS.columns[1:])]

# Assign Data
cum_confirmed = list(dfConfTS[dfConfTS["Country/Region"]==Country].values[0][day:])
cum_death = list(dfDeathTS[dfDeathTS["Country/Region"]==Country].values[0][day:])
cum_recovered = list(dfRecovTS[dfRecovTS["Country/Region"]==Country].values[0][day:])

# Output the visualization directly in the notebook
output_notebook()

# Create a figure with a datetime type x-axis
fig = figure(title='Cumulative Case of Covid-19 ('+str(dates_list[startDate(day)])+' until '+str(dates_list[-1])+')',
             plot_height=400, plot_width=700,
             x_axis_type ='datetime',
             x_axis_label='Dates', y_axis_label='Confirmed Case',
             x_minor_ticks=2, y_range=(0, max(cum_confirmed)),
             toolbar_location='above', tools = TOOLS)

# The daily case will be represented as vertical bars (columns)

fig.line(x=dates_list[day-1:], y=cum_confirmed,  
               color='blue', line_width=1, 
               legend_label='Confirmed')

fig.line(x=dates_list[day-1:], y=cum_death, 
               color='red', line_width=1, 
               legend_label='Death')

fig.line(x=dates_list[day-1:], y=cum_recovered, 
               color='green', line_width=1, 
               legend_label='Recovered')

fig.select_one(HoverTool).tooltips = [
    #('Dates ', '@x'),
    ('Number of People ', '@y'),
]

# Put the legend in the upper left corner
fig.legend.location = 'top_left'

# Let's check it out
print(f'Country : {Country}')
show(fig)

Country : Indonesia


# Make New DataFrame for Visualization 

In [284]:
# Per Country/Region DataFrame
def dailyReport(case):
  daily = []
  for idx in range(len(case)):
    if idx == 0:
      daily.append(case[idx])
    else:
      daily.append(case[idx]-case[idx-1])
  return daily

def makeDF(Country, stat):
  dates_list = [dt.strptime(date, "%m/%d/%y").date() for date in list(dfConfTS.columns[stat:])]
  cum_confirmed = list(dfConfTS[dfConfTS["Country/Region"]==Country].values[0][stat:])
  daily_confirmed = dailyReport(cum_confirmed)
  cum_death = list(dfDeathTS[dfDeathTS["Country/Region"]==Country].values[0][stat:])
  daily_death = dailyReport(cum_death)
  cum_recovered = list(dfRecovTS[dfRecovTS["Country/Region"]==Country].values[0][stat:])
  daily_recovered = dailyReport(cum_recovered)

  return pd.DataFrame({'Dates':dates_list, 'Confirmed':cum_confirmed, 'Death':cum_death, 'Recovered':cum_recovered, \
                       'dailyConfirmed':daily_confirmed, 'dailyDeath':daily_death, 'dailyRecovered':daily_recovered})

# Choose 1 for get all data
day = 1
Visualize_df = makeDF("Indonesia", day)
Visualize_df.tail()

,Dates,Confirmed,Death,Recovered,dailyConfirmed,dailyDeath,dailyRecovered
91,2020-04-22,7418,635,913,283,19,71
92,2020-04-23,7775,647,960,357,12,47
93,2020-04-24,8211,689,1002,436,42,42
94,2020-04-25,8607,720,1042,396,31,40
95,2020-04-26,8882,743,1107,275,23,65


In [285]:
def makeWorldDF(stat):
  dates_list = [dt.strptime(date, "%m/%d/%y").date() for date in list(dfConfTS.columns[stat:])]
  world_cum_confirmed = []
  world_cum_death = []
  world_cum_recovered = []
  for col in dfConfTS.columns[stat:]:
    world_cum_confirmed.append(sum(dfConfTS[col].values))
    world_cum_death.append(sum(dfDeathTS[col].values))
    world_cum_recovered.append(sum(dfRecovTS[col].values))
  world_daily_confirmed = dailyReport(world_cum_confirmed)
  world_daily_death = dailyReport(world_cum_death)
  world_daily_recovered = dailyReport(world_cum_recovered)

  return pd.DataFrame({'Dates':dates_list, 'Confirmed':world_cum_confirmed, 'Death':world_cum_death, 'Recovered':world_cum_recovered, \
                       'dailyConfirmed':world_daily_confirmed, 'dailyDeath':world_daily_death, 'dailyRecovered':world_daily_recovered})

days = 1  
World_visualize_df = makeWorldDF(days)
World_visualize_df.tail()

,Dates,Confirmed,Death,Recovered,dailyConfirmed,dailyDeath,dailyRecovered
91,2020-04-22,2623049,183064,710502,74958,6482,30112
92,2020-04-23,2707728,190857,739409,84679,7793,28907
93,2020-04-24,2809499,197004,790081,101771,6147,50672
94,2020-04-25,2896744,202844,817838,87245,5840,27757
95,2020-04-26,2971475,206542,865733,74731,3698,47895


# Visualize Data using World_visualize_df

In [286]:
# Create a figure with a datetime type x-axis
fig = figure(title='Daily Case of Covid-19 ('+str(dates_list[startDate(days)])+' until '+str(dates_list[-1])+')',
             plot_height=400, plot_width=700,
             x_axis_type ='datetime',
             x_axis_label='Dates', y_axis_label='Confirmed Case',
             x_minor_ticks=2, y_range=(0, max([max(World_visualize_df.Confirmed),max(World_visualize_df.Death),max(World_visualize_df.Recovered)])),
             toolbar_location='above', tools = TOOLS)

# The daily case will be represented as vertical bars (columns)

fig.line(x=World_visualize_df.Dates, y=World_visualize_df.Confirmed,  
               color='blue', line_width=1, 
               legend_label='Confirmed')

fig.line(x=World_visualize_df.Dates, y=World_visualize_df.Death, 
               color='red', line_width=1, 
               legend_label='Death')

fig.line(x=World_visualize_df.Dates, y=World_visualize_df.Recovered, 
               color='green', line_width=1, 
               legend_label='Recovered')

fig.select_one(HoverTool).tooltips = [
    #('Dates ', '@x'),
    ('Number of People ', '@y'),
]

# Put the legend in the upper left corner
fig.legend.location = 'top_left'

# Let's check it out
print(f'GLOBAL')
show(fig)

GLOBAL


In [287]:
# Create a figure with a datetime type x-axis
fig = figure(title='Daily Case of Covid-19 ('+str(dates_list[startDate(days)])+' until '+str(dates_list[-1])+')',
             plot_height=400, plot_width=700,
             x_axis_type ='datetime',
             x_axis_label='Dates', y_axis_label='Confirmed Case',
             x_minor_ticks=2, y_range=(0, max([max(World_visualize_df.dailyConfirmed),max(World_visualize_df.dailyDeath),max(World_visualize_df.dailyRecovered)])),
             toolbar_location='above', tools = TOOLS)

# The daily case will be represented as vertical bars (columns)

fig.line(x=World_visualize_df.Dates, y=World_visualize_df.dailyConfirmed,  
               color='blue', line_width=1, 
               legend_label='Confirmed')

fig.line(x=World_visualize_df.Dates, y=World_visualize_df.dailyDeath, 
               color='red', line_width=1, 
               legend_label='Death')

fig.line(x=World_visualize_df.Dates, y=World_visualize_df.dailyRecovered, 
               color='green', line_width=1, 
               legend_label='Recovered')

fig.select_one(HoverTool).tooltips = [
    #('Dates ', '@x'),
    ('Number of People ', '@y'),
]

# Put the legend in the upper left corner
fig.legend.location = 'top_left'

# Let's check it out
print(f'GLOBAL')
show(fig)

GLOBAL


# Visualize Data using Visualize_df

In [288]:
# Create a figure with a datetime type x-axis
fig = figure(title='Daily Case of Covid-19 ('+str(dates_list[startDate(day)])+' until '+str(dates_list[-1])+')',
             plot_height=400, plot_width=700,
             x_axis_type ='datetime',
             x_axis_label='Dates', y_axis_label='Confirmed Case',
             x_minor_ticks=2, y_range=(0, max([max(Visualize_df.dailyConfirmed),max(Visualize_df.dailyDeath),max(Visualize_df.dailyRecovered)])),
             toolbar_location='above', tools = TOOLS)

# The daily case will be represented as vertical bars (columns)

fig.line(x=Visualize_df.Dates, y=Visualize_df.dailyConfirmed,  
               color='blue', line_width=1, 
               legend_label='Confirmed')

fig.line(x=Visualize_df.Dates, y=Visualize_df.dailyDeath, 
               color='red', line_width=1, 
               legend_label='Death')

fig.line(x=Visualize_df.Dates, y=Visualize_df.dailyRecovered, 
               color='green', line_width=1, 
               legend_label='Recovered')

fig.select_one(HoverTool).tooltips = [
    #('Dates ', '@x'),
    ('Number of People ', '@y'),
]

# Put the legend in the upper left corner
fig.legend.location = 'top_left'

# Let's check it out
print(f'Country : {Country}')
show(fig)

Country : Indonesia


In [289]:
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'

output_notebook()

fig = figure(title='Cumulative Case of Covid-19 ('+str(dates_list[startDate(day)])+' until '+str(dates_list[-1])+')',
             plot_height=400, plot_width=700,
             x_axis_type ='datetime',
             x_axis_label='Dates', y_axis_label='Confirmed Case',
             x_minor_ticks=2, y_range=(0, max(Visualize_df.dailyConfirmed)),
             toolbar_location='above', tools = TOOLS)

fig.vbar(x=Visualize_df.Dates, top=Visualize_df.dailyConfirmed, width=0.9, legend_label='Confirmed')

fig.select_one(HoverTool).tooltips = [
    #('Dates ', '@x'),
    ('Number of People ', '@y'),
]

fig.legend.location = 'top_left'

print(f'Country : {Country}')
show(fig)

Country : Indonesia


# Link Plots (Confirmed, Death, Recovered)

In [303]:
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'

# linked plot 1
linked_plot1 = figure(width=250, height=250, x_axis_type ='datetime', title = 'Daily Confirmed', tools = TOOLS)
linked_plot1.circle(World_visualize_df.Dates, World_visualize_df.dailyConfirmed)

# create new plots and share both ranges
linked_plot2 = figure(width=250, height=250, x_range=linked_plot1.x_range, y_range=linked_plot1.y_range, x_axis_type ='datetime', title = 'Daily Death', tools = TOOLS)
linked_plot2.line(World_visualize_df.Dates, World_visualize_df.dailyDeath)

linked_plot3 = figure(width=250, height=250, x_range=linked_plot1.x_range, y_range=linked_plot1.y_range, x_axis_type ='datetime', title = 'Daily Recovered', tools = TOOLS)
linked_plot3.vbar(World_visualize_df.Dates, top=World_visualize_df.dailyRecovered, width=0.5)

# the subplots in a gridplot
linked_gridplot = gridplot([[linked_plot1, linked_plot2, linked_plot3]])

linked_plot1.select_one(HoverTool).tooltips = [
    ('Number of People ', '@y'),
]
linked_plot2.select_one(HoverTool).tooltips = [
    ('Number of People ', '@y'),
]
linked_plot3.select_one(HoverTool).tooltips = [
    ('Number of People ', '@y'),
]

# show the results
show(linked_gridplot)

# Covid-19 Case Map Plotting

In [319]:
# convert longitude, latitude to mercator coordinates
# example - Detroit Michigan 42.334197, -83.047752
# Above is Lat, Long

from pyproj import Proj, transform

def create_coordinates(long_arg,lat_arg):
    in_wgs = Proj(init='epsg:4326')
    out_mercator = Proj(init='epsg:3857')
    long, lat = long_arg, lat_arg
    mercator_x, mercator_y = transform(in_wgs, out_mercator, long, lat)
    return [mercator_x, mercator_y]
    #x dan y
    
# Detroit
create_coordinates(-83.047752,42.334197)

[-9244833.464166068, 5211172.739903524]

In [327]:
from bokeh.tile_providers import get_provider, Vendors

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
# tile_provider = get_provider(Vendors.STAMEN_TONER_BACKGROUND)

# range bounds supplied in web mercator coordinates
m = figure(plot_width=800, 
           plot_height=400,
           x_range=(-12000000, 9000000), 
           y_range=(-1000000, 7000000), 
           x_axis_type='mercator', 
           y_axis_type='mercator')

m.add_tile(tile_provider)

#for i in range(CovDetail.shape[0]): -> If wanna plot all, but it's too heavy. So let's give an example
for i in range(100):
  point = create_coordinates(CovDetail['Long_'].values[i],CovDetail['Lat'].values[i])
  m.circle(x=point[0], y=point[1], size=5, color = 'red')
  #m.circle(x=point[0], y=point[1], size=CovDetail['Confirmed'].values[i], color = 'red') -> If wanna use size change

show(m)

# Interactive Widgets

In [0]:
import ipywidgets
from bokeh.io import push_notebook
from bokeh.models import Range1d

In [329]:
dfCovid.head()

,Country,Confirmed,Deaths,Recovered,Latitude,Longitude,Death Rate(%),Not Recovered
0,Australia,6714,83,5541,-25.0000,133.0000,1.24,1090
1,Austria,15274,549,12362,47.5162,14.5501,3.59,2363
2,Canada,47147,2663,16883,60.0010,-95.0010,5.65,27601
3,China,83912,4637,78290,30.5928,114.3055,5.53,985
4,Denmark,8896,422,5994,56.0000,10.0000,4.74,2480


In [344]:
# widget
drop_down = ipywidgets.Dropdown(options=dfCovid['Country'].values,
                                value='Indonesia',
                                description='Country :',
                                disabled=False)

# data
x_bar_data_ipyw = ['Confirmed', 'Deaths', 'Recovered']
y_bar_data_ipyw = [dfCovid[dfCovid.Country=='Indonesia']['Confirmed'].values[0], 
                   dfCovid[dfCovid.Country=='Indonesia']['Deaths'].values[0], 
                   dfCovid[dfCovid.Country=='Indonesia']['Recovered'].values[0]]
    
# figure and plot
bar_chart_interactive = figure(x_range=x_bar_data_ipyw, plot_height=300)
bar_ipyw = bar_chart_interactive.vbar(x_bar_data_ipyw, top=y_bar_data_ipyw, color='green', width=0.5)
bar_chart_interactive.y_range=Range1d(0, 1000)

# function - bar chart
def country_case(cntry):
    if cntry in dfCovid.Country.values:
      bar_ipyw.data_source.data['top'] = [dfCovid[dfCovid.Country==cntry]['Confirmed'].values[0], 
                                          dfCovid[dfCovid.Country==cntry]['Deaths'].values[0], 
                                          dfCovid[dfCovid.Country==cntry]['Recovered'].values[0]]
    push_notebook()

show(bar_chart_interactive, notebook_handle=True)
        
# interaction
ipywidgets.interact(country_case, cntry=drop_down)

interactive(children=(Dropdown(description='Country :', index=88, options=('Australia', 'Austria', 'Canada', '…

<function __main__.country_case>